In [29]:
import os
import shutil
import dataiku
import pandas as pd

from dataiku import recipe

In [6]:
client = dataiku.api_client()
project = client.get_default_project()

In [13]:
# get train dataset
train_dataset = dataiku.Dataset("training_data")
evaluation_dataset = dataiku.Dataset("eval_data")

In [15]:
# get output saved model
sm = project.get_saved_model("VdHxdbkg")

# get train dataset as a pandas dataframe
df = train_dataset.get_dataframe()

In [18]:
# get the path of a local managed folder where to temporarily save the trained model
mf = dataiku.Folder("cTcsiBqc")
path = mf.get_path()

In [19]:
model_subdir = "my_subdir"
model_dir = os.path.join(path, model_subdir)

In [0]:
>>> from sklearn.datasets import load_iris
>>> from sklearn.linear_model import LogisticRegression
>>> X, y = load_iris(return_X_y=True)
>>> clf = LogisticRegression(random_state=0).fit(X, y)
>>> clf.predict(X[:2, :])
array([0, 0])
>>> clf.predict_proba(X[:2, :])
array([[9.8...e-01, 1.8...e-02, 1.4...e-08],
       [9.7...e-01, 2.8...e-02, ...e-08]])
>>> clf.score(X, y)
0.97...

In [20]:
from sklearn.linear_model import LogisticRegression

In [25]:
y = df['y']
X = df.drop('y', axis=1)

In [31]:
job_df = pd.get_dummies(df.job, prefix='job')
marital_df = pd.get_dummies(df.marital, prefix='marital')
education_df = pd.get_dummies(df.education, prefix='education')

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0


In [33]:
df.replace({"no": 5})

0          5
1          5
2          5
3          5
4          5
        ... 
31643    nov
31644    nov
31645    nov
31646    nov
31647    nov
Name: month, Length: 31648, dtype: object

In [27]:
X

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign
0,58,management,married,tertiary,no,2143,yes,no,5,may,261,1
1,44,technician,single,secondary,no,29,yes,no,5,may,151,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,76,1
3,58,retired,married,primary,no,121,yes,no,5,may,50,1
4,43,technician,single,secondary,no,593,yes,no,5,may,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...
31643,25,technician,single,secondary,no,505,no,yes,17,nov,386,2
31644,51,technician,married,tertiary,no,825,no,no,17,nov,977,3
31645,72,retired,married,secondary,no,5715,no,no,17,nov,1127,5
31646,57,blue-collar,married,secondary,no,668,no,no,17,nov,508,4


In [26]:
clf = LogisticRegression(random_state=0).fit(X, y)

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'nov'

In [0]:
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

try:
    # ...train your model...
    clf = LogisticRegression(random_state=0).fit(X, y)
    
    # ...save it with package specific MLflow method (here, SKlearn)...
    mlflow.sklearn.save_model(my_model, model_dir)

    # import the model, creating a new version
    mlflow_version = sm.import_mlflow_version_from_managed_folder("version_name", "local_managed_folder", model_subdir, "code-env-with-mlflow-name")
finally:
    shutil.rmtree(model_dir)

# setting metadata (target name, classes,...)
mlflow_version.set_core_metadata(target_column, ["class0", "class1",...] , get_features_from_dataset=evaluation_dataset.name)

# evaluate the performance of this new version, to populate the performance screens of the saved model version in DSS
mlflow_version.evaluate(evaluation_dataset.name)

In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
training_data = dataiku.Dataset("training_data")
training_data_df = training_data.get_dataframe()




# Write recipe outputs
MLflow_model = dataiku.Folder("cTcsiBqc")
MLflow_model_info = MLflow_model.get_info()